In [ ]:
import scipy.stats as sps
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import gc
from dateutil.parser import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import aiohttp
import asyncio
from datetime import datetime
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')
sns.set(font_scale=0)
from ipwhois import IPWhois
import socket
%matplotlib inline

In [ ]:
def get_host_owner(domain_name):
    try:
        ip = socket.gethostbyname(domain_name)
        obj = IPWhois(ip)
        results = obj.lookup_rdap(depth=0)
        return results['asn_description']
    except Exception:
        return "-"

In [ ]:
date = lib.execute_query("""
SELECT
    addMonths(toStartOfMonth(toDate(NOW())), -1) as dt
FORMAT TabSeparatedWithNames
""")['dt'].iloc[0]


In [ ]:
date

In [ ]:
req = f"""
SELECT
    StartURLDomain,
    cnt,
    final_count_id as count_id
FROM (
    SELECT
        StartURLDomain,
        SUM(cnt_raw) as cnt,
        argMax(count_id, cnt_raw) as raw_count_id,
        max(cnt_raw) as max_cnt,
        if(max_cnt > 0.5 * cnt, raw_count_id, null) as final_count_id
    FROM (
        SELECT
            StartURLDomain,
            CounterID as count_id,
            SUM(Sign) * 100 as cnt_raw
        FROM visits_all SAMPLE 1/100
        WHERE toStartOfMonth(toDate(StartDate)) >= toStartOfMonth(toDate('{date}'))
        AND IsRobot == 0
        AND IsYandex == 0
        AND isNotNull(YandexLogin)
        AND YandexLogin != ''
        GROUP BY StartURLDomain, count_id
        ORDER BY cnt_raw DESC
        LIMIT 200000
    )
    GROUP BY StartURLDomain
)
WHERE isNotNull(count_id)
ORDER BY cnt DESC
LIMIT 100000
FORMAT TabSeparatedWithNames
"""

In [ ]:
df = lib.get_metrika_df(req)

In [ ]:
from joblib import Parallel, delayed

In [ ]:
%%time
clouds = Parallel(n_jobs=-1)(delayed(get_host_owner)(domain) 
                            for domain in df["StartURLDomain"])

In [ ]:
df

In [ ]:
df["cloud_hosts"] = clouds

In [ ]:
df.to_csv("host_owners", index=False)

In [ ]:
hosts = pd.DataFrame(df.groupby(["cloud_hosts"]).size().sort_values(ascending=False), 
                     columns = ["cnt"])

In [ ]:
google = ["google"]
amazon = ["amazon"]
azure = ["microsoft"]
yandex = ["yandex"]
cloudflare = ["cloudflare"]
hetzner = ['hetzner']
ovh = ['ovh']
timeweb = ['timeweb']
as_reg = ['as-reg']
beget = ['beget']
firstvds = ['thefirst']
digitalocean = ['digitalocean']
dataline = ['dtln']
selectel = ['selectel']
ru_center = ['ru-center']
ihc = ['ihcru']
firstcolo = ['firstcolo']
mail = ['mailru']
rostelecom = ['rostelecom']
qrator = ['qrator']
belpak = ['belpak']
rambler = ['rambler']
sprinthost = ['sprinthost']

In [ ]:
clouds = ['google', 'amazon', 'azure', 'yandex', 'cloudflare',
          'hetzner', 'ovh', 'timeweb', 'as_reg', 'beget',
          'firstvds', 'digitalocean', 'dataline', 'selectel',
          'ru_center', 'ihc', 'firstcolo', 'mail', 'rostelecom', 
          'qrator', 'belpak', 'rambler', 'sprinthost']

In [ ]:
def host_finder(x):
    try:
        for cloud in clouds:
            for word in globals()[cloud]:
                if word in x.lower():
                    return cloud
    except Exception:
        pass
    return '-'

In [ ]:
#hosts_copy["cloud_hosts"] = hosts_copy.index
#hosts_copy["cloud_hosts"] = hosts_copy["cloud_hosts"].apply(lambda x: host_finder(x))

In [ ]:
df["cloud_host_name"] = df['cloud_hosts'].apply(lambda x: host_finder(x))

In [ ]:
df.columns = ['domain', 'visits_num', 'cloud_host', 'cloud_host_name']

In [ ]:
res = df[df["cloud_host_name"] != '-']

In [ ]:
res = pd.concat([res, pd.get_dummies(res["cloud_host_name"])], axis=1)

In [ ]:
path = "//home/metrika/export/code"
tables = []
for table in lib.find_tables_in_hahn_folder(path):
    if "exp." not in str(table):
        tables.append(table)

In [ ]:
path = str(tables[-1])

In [ ]:
path

In [ ]:
request = f"""
SELECT
    DISTINCT domain
From "{path}"
WHERE has_metrika_aggr = 1
FORMAT TabSeparatedWithNames
"""

In [ ]:
good_domains = lib.execute_query(request)

In [ ]:
domains = set(good_domains["domain"])

In [ ]:
res = res[res["domain"].isin(domains)]

In [ ]:
res.shape

-------

In [ ]:
import gc
gc.collect()

In [ ]:
owner_req = f"""
SELECT
    DISTINCT owner,
    email,
    site
FROM "//home/metrika/export/counters"
FORMAT TabSeparatedWithNames
"""

In [ ]:
owners = lib.execute_query(owner_req)

In [ ]:
owners_cleaned = owners[owners['site'].isin(res['domain'])]

In [ ]:
owners_cleaned.columns = ['puid', 'email', 'domain']

In [ ]:
owners_cleaned['puid']  = owners_cleaned['puid'].astype(int)

In [ ]:
final_df = pd.merge(res, owners_cleaned, on = "domain")

In [ ]:
final_df = final_df[final_df['cloud_host_name'] != 'yandex']

In [ ]:
final_df['amazon'].dtype == np.dtype('uint8')

In [ ]:
for col in final_df.columns:
    if final_df[col].dtype == np.dtype('uint8'):
        final_df[col] = final_df[col].astype(int)

In [ ]:
bill_df = lib.execute_query("""
SELECT
    puid,
    argMax(billing_account_id, event_time) as billing_account_id
FROM "//home/cloud_analytics/cubes/acquisition_cube/cube"
WHERE event == 'ba_created'
GROUP BY puid
FORMAT TabSeparatedWithNames
"""
)

In [ ]:
final_df['puid'] = final_df['puid'].astype(str)
bill_df['puid'] = bill_df['puid'].astype(str)
final_df.replace('nan', np.nan, inplace=True)
final_df_with_bill = pd.merge(final_df, bill_df, on='puid', how='left')

In [ ]:
lib.save_table('2020-08-01', '//home/cloud_analytics/lunin-dv/competitors_host_owners',
               final_df_with_bill)